<a href="https://colab.research.google.com/github/balams81/Rain/blob/master/Parameter_tuning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from datetime import datetime

Using TensorFlow backend.


In [3]:
from google.colab import files
uploaded = files.upload()

Saving lstmdata.csv to lstmdata.csv


In [0]:
data_frame = pd.read_csv('lstmdata.csv')
df_raw = data_frame.iloc[260000:270000,1:-1].values
rain_raw = data_frame.iloc[260000:270000,7:8].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
df = sc.fit_transform(df_raw)

sc1 = MinMaxScaler(feature_range = (0, 1))
rain = sc1.fit_transform(rain_raw)

In [0]:
np.random.seed(5)
le = LabelEncoder()
X = np.array(df)
y = np.array(rain)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [0]:
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, activation='relu'))
	model.add(Dense(8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [0]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=10, verbose=0)

In [8]:
model.fit(X_train, y_train)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [0]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model)

In [10]:
batch_sizes = [10, 20, 50, 100]
epochs = [5, 10, 50]
parameters = {'batch_size': batch_sizes, 'epochs': epochs}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/5
4666/4666 [==============================] - 2s 380us/step - loss: 0.5171 - acc: 0.9522
Epoch 2/5
4666/4666 [==============================] - 2s 325us/step - loss: 0.4700 - acc: 0.9522
Epoch 3/5
4666/4666 [==============================] - 2s 338us/step - loss: 0.4559 - acc: 0.9522
Epoch 4/5
4666/4666 [==============================] - 2s 338us/step - loss: 0.4428 - acc: 0.9522
Epoch 5/5
2334/2334 [==============================] - 0s 133us/step
Epoch 1/5
4667/4667 [==============================] - 2s 397us/step - loss: 0.4497 - acc: 0.9490
Epoch 2/5
4667/4667 [==============================] - 2s 332us/step - loss: 0.4121 - acc: 0.9490
Epoch 3/5
4667/4667 [==============================] - 2s 326us/step - loss: 0.4020 - acc: 0.9490
Epoch 4/5
4667/4667 [==============================] - 2s 329us/step - loss: 0.3862 - acc: 0.9490
Epoch 5/5
2333/2333 [==============================] - 0s 150us/step
Epoch 1/5
4667/4667 [==============================] - 2s 386us/step - loss: 0

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fc94d37a7b8>,
             iid='warn', n_jobs=None,
             param_grid={'batch_size': [10, 20, 50, 100],
                         'epochs': [5, 10, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [11]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.9511428586414882 {'batch_size': 100, 'epochs': 5}
0.950999992958137 {'batch_size': 10, 'epochs': 5}
0.8932857080016817 {'batch_size': 10, 'epochs': 10}
0.868999992438725 {'batch_size': 10, 'epochs': 50}
0.9511428497093064 {'batch_size': 20, 'epochs': 5}
0.942142849947725 {'batch_size': 20, 'epochs': 10}
0.9284285655617714 {'batch_size': 20, 'epochs': 50}
0.9511428569384983 {'batch_size': 50, 'epochs': 5}
0.950714285492897 {'batch_size': 50, 'epochs': 10}
0.9102857112373625 {'batch_size': 50, 'epochs': 50}
0.9511428586414882 {'batch_size': 100, 'epochs': 5}
0.9511428586414882 {'batch_size': 100, 'epochs': 10}
0.8980000037040029 {'batch_size': 100, 'epochs': 50}


In [18]:
def create_model(optimizer):
    model = Sequential()
    model.add(Dense(12, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='softmax'))
    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100)
parameters = {'optimizer':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/10
4666/4666 [==============================] - 7s 1ms/step - loss: 1.0994 - acc: 0.0225
Epoch 2/10
4666/4666 [==============================] - 0s 32us/step - loss: 1.0994 - acc: 0.0225
Epoch 3/10
4666/4666 [==============================] - 0s 32us/step - loss: 1.0994 - acc: 0.0225
Epoch 4/10
4666/4666 [==============================] - 0s 30us/step - loss: 1.0994 - acc: 0.0225
Epoch 5/10
4666/4666 [==============================] - 0s 29us/step - loss: 1.0994 - acc: 0.0225
Epoch 6/10
4666/4666 [==============================] - 0s 31us/step - loss: 1.0994 - acc: 0.0225
Epoch 7/10
4666/4666 [==============================] - 0s 35us/step - loss: 1.0994 - acc: 0.0225
Epoch 8/10
4666/4666 [==============================] - 0s 31us/step - loss: 1.0994 - acc: 0.0225
Epoch 9/10
4666/4666 [==============================] - 0s 32us/step - loss: 1.0994 - acc: 0.0225
Epoch 10/10
2334/2334 [==============================] - 3s 1ms/step
Epoch 1/10
4667/4667 [============================

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fc8d5aa2d68>,
             iid='warn', n_jobs=None,
             param_grid={'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta',
                                       'Adam', 'Adamax', 'Nadam']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [19]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.023428570976746933 {'optimizer': 'SGD'}
0.023428570976746933 {'optimizer': 'SGD'}
0.023428570976746933 {'optimizer': 'RMSprop'}
0.023428570976746933 {'optimizer': 'Adagrad'}
0.023428570976746933 {'optimizer': 'Adadelta'}
0.023428570976746933 {'optimizer': 'Adam'}
0.023428570976746933 {'optimizer': 'Adamax'}
0.023428570976746933 {'optimizer': 'Nadam'}


In [21]:
def create_model(activation):
    model = Sequential()
    model.add(Dense(12, activation=activation))
    model.add(Dense(8, activation=activation))
    model.add(Dense(1, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32)
parameters = {'activation':['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/50
4666/4666 [==============================] - 8s 2ms/step - loss: nan - acc: 0.0225
Epoch 2/50
4666/4666 [==============================] - 1s 131us/step - loss: nan - acc: 0.0225
Epoch 3/50
4666/4666 [==============================] - 1s 127us/step - loss: nan - acc: 0.0225
Epoch 4/50
4666/4666 [==============================] - 1s 125us/step - loss: nan - acc: 0.0225
Epoch 5/50
4666/4666 [==============================] - 1s 128us/step - loss: nan - acc: 0.0225
Epoch 6/50
4666/4666 [==============================] - 1s 125us/step - loss: nan - acc: 0.0225
Epoch 7/50
4666/4666 [==============================] - 1s 126us/step - loss: nan - acc: 0.0225
Epoch 8/50
4666/4666 [==============================] - 1s 125us/step - loss: nan - acc: 0.0225
Epoch 9/50
4666/4666 [==============================] - 1s 125us/step - loss: nan - acc: 0.0225
Epoch 10/50
4666/4666 [==============================] - 1s 128us/step - loss: nan - acc: 0.0225
Epoch 11/50
4666/4666 [==================

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fc8d45e0e80>,
             iid='warn', n_jobs=None,
             param_grid={'activation': ['softmax', 'softplus', 'softsign',
                                        'relu', 'tanh', 'sigmoid',
                                        'hard_sigmoid', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [22]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.02342857143495764 {'activation': 'softmax'}
0.02342857143495764 {'activation': 'softmax'}
0.02342857143495764 {'activation': 'softplus'}
0.02342857143495764 {'activation': 'softsign'}
0.02342857143495764 {'activation': 'relu'}
0.02342857143495764 {'activation': 'tanh'}
0.02342857143495764 {'activation': 'sigmoid'}
0.02342857143495764 {'activation': 'hard_sigmoid'}
0.02342857143495764 {'activation': 'linear'}
